In [53]:
%matplotlib inline

from Bio.Seq import Seq
from Bio import SeqIO
from Bio.Alphabet import IUPAC
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.ensemble import RandomForestClassifier

In [54]:
pd.set_option('display.max_rows', 500)

# Importar dados
Importa os dados dos arquivos ".fasta" e coloca em `pd.DataFrame`.

Cada fasta contém sequências de ZIKV originárias de uma região geográfica:

- Africa
- Asia
- Oceania (incl. French Polynesia and Micronesia)
- Americas (south and north plus Europe)

Cada linha é uma amostra sequenciada.

Cada coluna (atributo) é uma posição na sequência genética com sua respectica base (acgt).

In [55]:
africa_fasta = '../DATA/!CLEAN/AFRICA_clear_v2_curated_Trimmed_Aligned_Z1_Z2_NCBI.fas'
asia_fasta = '../DATA/!CLEAN/ASIA_clear_v2_curated_Trimmed_Aligned_Z1_Z2_NCBI.fas'
oceania_fasta = '../DATA/!CLEAN/OCEANIA_clear_v2_curated_Trimmed_Aligned_Z1_Z2_NCBI.fas'
americas_fasta = '../DATA/!CLEAN/AMERICAS_clear_v2_curated_Trimmed_Aligned_Z1_Z2_NCBI.fas'
original_fasta = '../DATA/!CLEAN/clear_v2_curated_Trimmed_Aligned_Z1_Z2_NCBI.fas'


In [56]:
# Gets the sequences IDs
identifiers_africa = [seq_record.id for seq_record in SeqIO.parse(africa_fasta, "fasta")]
# Gets the sequences nucleotides
seqs_africa = np.array([list(str(seq_rec.seq)) for seq_rec in SeqIO.parse(africa_fasta, "fasta")])
# Creates columns names based on position, starting from 1, to make it easier to analyse results
cols = list(range(1, seqs_africa.shape[1]+1))

# Creates dataframe with data
df_africa = pd.DataFrame(seqs_africa, index=identifiers_africa, columns=cols)
#df_africa.head()

In [57]:
# Gets the sequences IDs
identifiers_asia = [seq_record.id for seq_record in SeqIO.parse(asia_fasta, "fasta")]
# Gets the sequences nucleotides
seqs_asia = np.array([list(str(seq_rec.seq)) for seq_rec in SeqIO.parse(asia_fasta, "fasta")])
# Creates columns names based on position, starting from 1, to make it easier to analyse results
cols = list(range(1, seqs_asia.shape[1]+1))

# Creates dataframe with data
df_asia = pd.DataFrame(seqs_asia, index=identifiers_asia, columns=cols)
#df_asia.head()

In [58]:
# Gets the sequences IDs
identifiers_oceania = [seq_record.id for seq_record in SeqIO.parse(oceania_fasta, "fasta")]
# Gets the sequences nucleotides
seqs_oceania = np.array([list(str(seq_rec.seq)) for seq_rec in SeqIO.parse(oceania_fasta, "fasta")])
# Creates columns names based on position, starting from 1, to make it easier to analyse results
cols = list(range(1, seqs_oceania.shape[1]+1))

# Creates dataframe with data
df_oceania = pd.DataFrame(seqs_oceania, index=identifiers_oceania, columns=cols)
#df_oceania.head()

In [59]:
# Gets the sequences IDs
identifiers_americas = [seq_record.id for seq_record in SeqIO.parse(americas_fasta, "fasta")]
# Gets the sequences nucleotides
seqs_americas = np.array([list(str(seq_rec.seq)) for seq_rec in SeqIO.parse(americas_fasta, "fasta")])
# Creates columns names based on position, starting from 1, to make it easier to analyse results
cols = list(range(1, seqs_americas.shape[1]+1))

# Creates dataframe with data
df_americas = pd.DataFrame(seqs_americas, index=identifiers_americas, columns=cols)
#df_americas.head()

In [60]:
# Gets the sequences IDs
identifiers_original = [seq_record.id for seq_record in SeqIO.parse(original_fasta, "fasta")]
# Gets the sequences nucleotides
seqs_original = np.array([list(str(seq_rec.seq)) for seq_rec in SeqIO.parse(original_fasta, "fasta")])
# Creates columns names based on position, starting from 1, to make it easier to analyse results
cols = list(range(1, seqs_original.shape[1]+1))

# Creates dataframe with data
df_original = pd.DataFrame(seqs_original, index=identifiers_original, columns=cols)
#df_original

# Vetor de "classes"
Anotar os dataframes acima, inserindo uma coluna 'class' contendo 0 ou 1.

Isso dá a flexibilidade de realizar diversas análises com esse dataset, bastando para isso mudar as classes.

In [61]:
df_americas['class'] = np.ones(len(df_americas))
# df_americas.head()

In [62]:
df_oceania['class'] = np.ones(len(df_oceania))
#df_oceania.tail()

In [63]:
df_asia['class'] = np.zeros(len(df_asia))
#df_asia.tail()

In [64]:
df_africa['class'] = np.zeros(len(df_africa))
#df_africa.tail()

# Criação do dataset para análise

Nesse ponto, juntamos os dataframes que serão utilizados na análise em um só dataframe.

In [65]:
df_1 = df_americas.append(df_oceania)
df_1 = df_1.append(df_asia)
df_1 = df_1.append(df_africa)

df_2 = df_original

In [66]:
#df_1

In [67]:
#df_2

# One-Hot Encoding
Aplicar o "one-hot encoding" para tratar os dados.

Como os dados genômicos são categóricos (ACGT), devem ser preparados de forma que fiquem adequados ao algoritmo de machine learning.

O método abaixo `pd.get_dummies(df)` transforma cada coluna com atributos categóricos (com "c" categorias, no nosso caso 4 categorias, ACGT) em "c" colunas contendo 0 em todas posições, exceto na posição correspondente ao valor presente naquele atributo.

Assim:

|sample|base1|base2|
|------|-----|-----|
|s_1   |A    |T    |
|s_2   |A    |G    |
|s_3   |C    |T    |

Se tornaria:

|sample|base1_A|base1_C|base1_G|base1_T|base2_A|base2_C|base2_G|base2_T|
|------|-------|-------|-------|-------|-------|-------|-------|-------|
|s_1   |1      |0      |0      |0      |0      |0      |0      |1      |
|s_2   |1      |0      |0      |0      |0      |0      |1      |0      |
|s_3   |0      |1      |0      |0      |0      |0      |0      |1      |

In [68]:
df_ohe_1 = pd.get_dummies(df_1)

In [69]:
df_ohe_2 = pd.get_dummies(df_2)

In [70]:
index_1 = df_ohe_1.index
#index_1

In [71]:
index_2 = df_ohe_2.index
#print(index_2)

In [72]:
classes = df_ohe_1["class"]
#classes

In [73]:
df_ohe_1 = df_ohe_1.drop('class', axis=1)

In [74]:
values_1 = df_ohe_1.values.astype(int)
values_2 = df_ohe_2.values.astype(int)
#values_1

In [75]:
#df_ohe_1.to_csv('matriz_1.csv')

In [76]:
#df_ohe_2.to_csv('matriz_2.csv')

In [77]:
list_index_1 = np.array(list(index_1))
list_index_1 = list_index_1.T
df_index_1 = pd.DataFrame(list_index_1)
values_3 = df_index_1.values

In [78]:
list_index_2 = np.array(list(index_2))
list_index_2 = list_index_2.T
df_index_2 = pd.DataFrame(list_index_2)
values_4 = df_index_2.values

In [79]:
classes = np.array(list(classes))
classes = classes.T
df_classes = pd.DataFrame(classes)
val_classes = df_classes.values.astype(int)

In [80]:
df_index_1.to_csv('matriz_3.csv')
df_index_2.to_csv('matriz_4.csv')


In [81]:
np.savetxt("matriz_1.csv", values_1, delimiter=" ", fmt='%1i')
np.savetxt("matriz_2.csv", values_2, delimiter=" ", fmt='%1i')
#np.savetxt("matriz_3.csv", values_3, delimiter=" ")
#np.savetxt("matriz_4.csv", values_4, delimiter=" ")
np.savetxt("matriz_5.csv", val_classes, delimiter=" ", fmt='%1i')